In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hr-analytics-job-change-of-data-scientists/sample_submission.csv
/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_test.csv
/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_train.csv


In [5]:
# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [9]:
df = pd.read_csv('/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_train.csv')

In [10]:
df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


So, we should predict, who is ready to be hunted :) <br/>


Avaliable train data description:
* enrollee_id : Unique ID for candidate
* city: City code
* city_ development _index : Developement index of the city (scaled)
* gender: Gender of candidate
* relevent_experience: Relevant experience of candidate
* enrolled_university: Type of University course enrolled if any
* education_level: Education level of candidate
* major_discipline :Education major discipline of candidate
* * experience: Candidate total experience in years
* * company_size: No of employees in current employer's company
* company_type : Type of current employer
* lastnewjob: Difference in years between previous job and current job
* training_hours: training hours completed
* target: 0 – Not looking for job change, 1 – Looking for a job change

In [11]:
Y = df.target

In [18]:
Y.value_counts()
df = df.drop('target', axis=1)

Look at the data - The most part of participants aren't ready to change his job :C <br/>

*Lets do a quick view of our features:*


In [20]:
for column in df.columns:
    print('Feature:', column)
    print(df[column].value_counts())

Feature: enrollee_id
6147     1
13612    1
7465     1
29988    1
23841    1
        ..
4759     1
6806     1
12947    1
14994    1
2047     1
Name: enrollee_id, Length: 19158, dtype: int64
Feature: city
city_103    4355
city_21     2702
city_16     1533
city_114    1336
city_160     845
            ... 
city_121       3
city_129       3
city_111       3
city_171       1
city_140       1
Name: city, Length: 123, dtype: int64
Feature: city_development_index
0.920    5200
0.624    2702
0.910    1533
0.926    1336
0.698     683
         ... 
0.824       4
0.807       4
0.625       3
0.781       3
0.664       1
Name: city_development_index, Length: 93, dtype: int64
Feature: gender
Male      13221
Female     1238
Other       191
Name: gender, dtype: int64
Feature: relevent_experience
Has relevent experience    13792
No relevent experience      5366
Name: relevent_experience, dtype: int64
Feature: enrolled_university
no_enrollment       13817
Full time course     3757
Part time course     119

**Which features are binary?**

Unfortunately the data doesn't provide us any prepeared binary features :( <br/>
Let's get them ourselves!
<p><b>Which features can be converted to binary?</b></p>

* Gender: "<b>isMale</b>"? Let's guess that men are less stable in his job and want to change it! This our first binary feature. *We will have some problems with "Others"..diversity chase us in HR world... They are not men - 0 for it.*
* relevent_experience: You can "Has relevent experience" or "No relevent experience". Without the third variant,  thats ok. "<b>HasRelevantExperience</b>" feature - our friend.
* enrolled_university: Keep it simple! For the first step we can take "<b>isEnrolled</b>" feature. What time? Full or Part? Just make it simple. 

**Which features are numerical?**
* training_hours
* city_development_index

*Thats all..Lets get some more*

**Which features can be converted to numerical?**

* education_level - we have to grade our categories of education
* experience - lets set 0 for "<1" and 21 for "20+".
* last_new_job - but what to do with "never"? Opened question. We will decide lateer


**Which features are categorical?**

* city - I guess "city index" is enough. Except that we can find "Favorite" city to relocate from.
* major_discipline
* company_size
* company_type

In [14]:
df.education_level.value_counts()

Graduate          11598
Masters            4361
High School        2017
Phd                 414
Primary School      308
Name: education_level, dtype: int64